In [122]:
import argparse
import logging
import sys

import monai
import torch
from torch.utils.tensorboard import SummaryWriter
from monai.utils import set_determinism
import numpy as np

from seg_data import getSegmentationDataset
from seg_model import getUNetForSegmentation, getUNETRForSegmentation

from transforms_dict import getSegmentationPostProcessingForLabel, getSegmentationPostProcessingForLabelOutput
from utils import compute_mean_dice, getReducePlateauScheduler, getAdamOptimizer, loadExistingModel
from utils import print_model_output, check_model_name, getDevice, getWorst, getBest
from seg_data import getSegmentationDataset
from seg_model import getUNetForSegmentation
from transforms_dict import getSegmentationPostProcessingForLabel, getSegmentationPostProcessingForLabelOutput
from utils import compute_mean_dice, getReducePlateauScheduler, getAdamOptimizer, loadExistingModel
from utils import print_model_output, check_model_name, getDevice

from monai.inferers import sliding_window_inference
from monai.data import decollate_batch
from monai.metrics import DiceMetric

In [123]:
#Parameters
dataset = "GIN"
verbose = True
augment = False
n4 = False

In [124]:
#Modelname and device
torch.multiprocessing.set_sharing_strategy('file_system')
set_determinism(seed=0)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
device = getDevice()

In [125]:
from monai.transforms import Activations
from torch import nn

#Loss
def loss_munet(preds, labels):
    dice = 1-torch.div(
        torch.sum(torch.mul(torch.mul(labels,preds),2)),
        torch.sum(torch.mul(preds,preds)) + torch.sum(torch.mul(labels,labels))
        )    
    return dice

loss_GDice = monai.losses.GeneralizedDiceLoss(other_act=nn.Softmax(dim=1))
loss_DiceCE = monai.losses.DiceCELoss(other_act=nn.Softmax(dim=1))

def loss_CE(input, target):
        n_pred_ch, n_target_ch = input.shape[1], target.shape[1]
        if n_pred_ch == n_target_ch:
            target = torch.argmax(target, dim=1)
        else:
            target = torch.squeeze(target, dim=1)
        target = target.long()
        return nn.CrossEntropyLoss(reduction="mean")(input, target)
    
def loss_GDiceCE(input, target, lambda_gdice=1.0, lambda_ce=0.5):    
    GDice = loss_GDice(input, target)
    CE = loss_CE(input, target)
    GDiceCELoss = lambda_gdice*GDice + lambda_ce*CE
    return GDiceCELoss

In [126]:
#Metric MUNet
def metric_munet(preds, labels):  
    labels = labels.detach().cpu().numpy()
    preds = preds.detach().cpu().numpy()    
    labels[np.where(labels == np.amax(labels, axis=0))] = 1
    labels[labels != 1] = 0
    dice=2*np.sum(labels*preds,(1,2,3))/(np.sum((labels+preds),(1,2,3))+1)    
    return dice

In [127]:
#postprocessing
outputs_processing = getSegmentationPostProcessingForLabelOutput(axis=1)
labels_processing = getSegmentationPostProcessingForLabel(axis=1)

#activations
softmax = Activations(other=nn.Softmax(dim=1))

In [128]:
#dataloaders
dataloaders, size = getSegmentationDataset(dataset=dataset, batch=1, augment=augment, training=False, n4=n4, labels=True, eval_augment=False)

=> Using GIN dataset.


Loading dataset: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 44779.05it/s]


In [129]:
import nibabel as nib
ptdr = "dataset3/Atlas/P56_Atlas_128_norm_id.nii.gz"
affine = nib.load(ptdr).affine
header = nib.load(ptdr).header

In [134]:
model = getUNetForSegmentation()
#model = getUNETRForSegmentation()
modelnames = [
    "old_final/seg/test_labels_dicece.pth",
    #"test_labels_unetr.pth"
    #"test_labels_femina3_dicece.pth",
    #"test_labels_femina3_gdice.pth",
    #"test_labels_femina3_gdicefoc.pth",
    #"test_labels_femina3_unetr_dicece.pth",
             ]

softmax = Activations(other=nn.Softmax(dim=1))
minn = 100

for modelname in modelnames:
    
    modelname = check_model_name(modelname)
    loadExistingModel(model, None, ft=modelname)
    print(modelname)
    model.eval()

    scores_list = []

    with torch.no_grad():
        metrics = [[], [], [], []]        
        phase = "test"
        for i, data in enumerate(dataloaders[phase]):
            if "4mois_6827" in data['img_meta_dict']['filename_or_obj'][0]:
                continue
            print("{}/{}".format(
                i, len(dataloaders[phase])), end='\r'
            )
            inputs, labels = data["img"].to(device), data["seg"].to(device)
            mask_filename = data['img_meta_dict']['filename_or_obj'][0].replace("/MRI/", "/Mask/")
            mask = torch.from_numpy(nib.load(mask_filename).get_fdata()).to(device).unsqueeze(0).unsqueeze(0)
            mask = (mask != 0)
            labels = labels * mask     
            onehot_labels = monai.networks.utils.one_hot(labels, num_classes=4,dim=1)
            
            filename = data['img_meta_dict']['filename_or_obj'][0].split('/')[-1]
            lbl_filename = data['seg_meta_dict']['filename_or_obj'][0].split('/')[-1]
            labels = labels.squeeze(2)
            
            outputs = sliding_window_inference(inputs, (128, 128, 128), 1, model)   
            probs = softmax(outputs) 
            probs_labels = torch.from_numpy(np.argmax(probs, axis=1)).unsqueeze(0).to(device) 
            probs_labels = probs_labels * mask        
            onehot_probs = monai.networks.utils.one_hot(probs_labels, num_classes=4,dim=1).to(device)
            
            #print(onehot_probs.shape)
            #print(onehot_probs[0,:,64,64,64]) 
            #print(onehot_labels.shape)
            #print(onehot_labels[0,:,64,64,64])
            
            preds = [outputs_processing(pred) for pred in decollate_batch(outputs)]
            labels = [label for label in decollate_batch(onehot_labels)]      
            
            dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)
            dice_metric(y_pred=onehot_probs.squeeze(), y=onehot_labels.squeeze())
            metric = dice_metric.aggregate()
            
            #if torch.mean(metric).item() < minn:
            #    minn = torch.mean(metric).item()
            #    print(minn)
            #    print(filename)
            
            name = "_".join(data['img_meta_dict']['filename_or_obj'][0].split('/')[-1].split('.')[0].split('_')).replace("_affine_","_")
            if "unetr" in modelname:
                lossname = "unetr"
            else:
                lossname = modelname.split('.')[0].split('_')[-1]            
            outputs_labels = outputs.detach().cpu().numpy()
            outputs_labels = np.argmax(outputs_labels, axis=1)
            outputs_labels = outputs_labels.squeeze()            
            mask = mask.squeeze().cpu().numpy()
            outputs_labels = outputs_labels * mask
            outputname = "dataset3/GIN/LabelsBackFromAtlas/" + name + "_from" + lossname + ".nii.gz"
            nib.save(nib.Nifti1Image(outputs_labels, affine, header), outputname)  
            print(outputname)
            
            for j in range(4):
                metrics[j].append(metric[j].item())   
   
        #running_loss /= size[phase]
        #print(metrics)
        mean_metrics = [np.mean(x) for x in metrics]
        std_metrics = [np.std(x) for x in metrics]
        model_mean_dice = np.mean(mean_metrics)
        print(
            "dice: {:.4f}".format(
                model_mean_dice
            )
        )
        print("dices: {:.3f}, {:.3f}, {:.3f}, {:.3f}".format(mean_metrics[0], mean_metrics[1], mean_metrics[2], mean_metrics[3]))
        print("stdss: {:.3f}, {:.3f}, {:.3f}, {:.3f}".format(std_metrics[0], std_metrics[1], std_metrics[2], std_metrics[3]))
        print('-'*10)

old_final/seg/test_labels_dicece.pth
dataset3/GIN/LabelsBackFromAtlas/M872_N4_o_128_ras_norm_id_fromdicece.nii.gz
dataset3/GIN/LabelsBackFromAtlas/M874_N4_o_128_ras_norm_id_fromdicece.nii.gz
dataset3/GIN/LabelsBackFromAtlas/M875_N4_o_128_ras_norm_id_fromdicece.nii.gz
dataset3/GIN/LabelsBackFromAtlas/M877_N4_o_128_ras_norm_id_fromdicece.nii.gz
dataset3/GIN/LabelsBackFromAtlas/M886_N4_o_128_ras_norm_id_fromdicece.nii.gz
dataset3/GIN/LabelsBackFromAtlas/M8876_N4_o_128_ras_norm_id_fromdicece.nii.gz
dice: 0.7291
dices: 0.995, 0.564, 0.933, 0.424
stdss: 0.001, 0.016, 0.004, 0.028
----------


In [ ]:
old_final/seg/test_labels_dicece.pth
dice: 0.7291
dices: 0.995, 0.564, 0.933, 0.424
stdss: 0.001, 0.016, 0.004, 0.028
    
-----------------------------------------


test_labels_unetr.pth
dice: 0.6570
dices: 0.994, 0.495, 0.918, 0.221
stdss: 0.002, 0.006, 0.003, 0.054

In [95]:
#model = getUNetForSegmentation()
model = getUNETRForSegmentation()
modelnames = [
    #"old_final/seg/test_labels_fce.pth",   
    #"old_final/seg/test_labels_gdice.pth",
    #"old_final/seg/test_labels_dicece.pth",
    #"test_labels_unetr.pth"
    #"test_labels_femina3_dicece.pth",
    #"test_labels_femina3_gdice.pth",
    #"test_labels_femina3_gdicefoc.pth",
    "test_labels_femina3_unetr_dicece.pth",
             ]

softmax = Activations(other=nn.Softmax(dim=1))
minn = 100

for modelname in modelnames:
    
    modelname = check_model_name(modelname)
    loadExistingModel(model, None, ft=modelname)
    print(modelname)
    model.eval()

    scores_list = []

    with torch.no_grad():
        metrics = [[], [], [], []]        
        phase = "test"
        for i, data in enumerate(dataloaders[phase]):
            if "4mois_6827" in data['img_meta_dict']['filename_or_obj'][0]:
                continue
            print("{}/{}".format(
                i, len(dataloaders[phase])), end='\r'
            )
            inputs, labels = data["img"].to(device), data["seg"].to(device)
            mask_filename = data['img_meta_dict']['filename_or_obj'][0].replace("/MRI/", "/Labels2/")
            mask = torch.from_numpy(nib.load(mask_filename).get_fdata()).to(device).unsqueeze(0).unsqueeze(0)
            mask = (mask != 0)
            labels = labels * mask     
            onehot_labels = monai.networks.utils.one_hot(labels, num_classes=4,dim=1)
            
            filename = data['img_meta_dict']['filename_or_obj'][0].split('/')[-1]
            lbl_filename = data['seg_meta_dict']['filename_or_obj'][0].split('/')[-1]
            labels = labels.squeeze(2)
            
            outputs = sliding_window_inference(inputs, (128, 128, 128), 1, model)   
            probs = softmax(outputs) 
            probs_labels = torch.from_numpy(np.argmax(probs, axis=1)).unsqueeze(0).to(device) 
            probs_labels = probs_labels * mask        
            onehot_probs = monai.networks.utils.one_hot(probs_labels, num_classes=4,dim=1).to(device)
            
            #print(onehot_probs.shape)
            #print(onehot_probs[0,:,64,64,64]) 
            #print(onehot_labels.shape)
            #print(onehot_labels[0,:,64,64,64])
            
            preds = [outputs_processing(pred) for pred in decollate_batch(outputs)]
            labels = [label for label in decollate_batch(onehot_labels)]      
            
            dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)
            dice_metric(y_pred=onehot_probs.squeeze(), y=onehot_labels.squeeze())
            metric = dice_metric.aggregate()
            
            if torch.mean(metric).item() < minn:
                minn = torch.mean(metric).item()
                print(minn)
                print(filename)
            
            name = "_".join(data['img_meta_dict']['filename_or_obj'][0].split('/')[-1].split('.')[0].split('_')).replace("_affine_","_")
            if "unetr" in modelname:
                lossname = "unetr"
            else:
                lossname = modelname.split('.')[0].split('_')[-1]
            
            #outputs_labels = outputs.detach().cpu().numpy()
            #outputs_labels = np.argmax(outputs_labels, axis=1)
            #outputs_labels = outputs_labels.squeeze()            
            #mask = mask.squeeze().cpu().numpy()
            #outputs_labels = outputs_labels * mask
            #outputname = "dataset3/Femina3/LabelsBackFromAtlas/" + name + "_from" + lossname + ".nii.gz"
            #nib.save(nib.Nifti1Image(outputs_labels, affine, header), outputname)  
            #print(outputname)
            
            for j in range(4):
                metrics[j].append(metric[j].item())   
   
        #running_loss /= size[phase]
        #print(metrics)
        mean_metrics = [np.mean(x) for x in metrics]
        std_metrics = [np.std(x) for x in metrics]
        model_mean_dice = np.mean(mean_metrics)
        print(
            "dice: {:.4f}".format(
                model_mean_dice
            )
        )
        print("dices: {:.3f}, {:.3f}, {:.3f}, {:.3f}".format(mean_metrics[0], mean_metrics[1], mean_metrics[2], mean_metrics[3]))
        print("stdss: {:.3f}, {:.3f}, {:.3f}, {:.3f}".format(std_metrics[0], std_metrics[1], std_metrics[2], std_metrics[3]))
        print('-'*10)

test_labels_femina3_unetr_dicece.pth
0.7205321788787842
2mois_7428_dki_F_N4_128_ras_norm_id.nii.gz
0.6878746747970581
4mois_7445_dki_M_N4_128_ras_norm_id.nii.gz
0.6539582014083862
4mois_7457_wt_F_N4_128_ras_norm_id.nii.gz
dice: 0.7438
dices: 0.998, 0.934, 0.688, 0.355
stdss: 0.001, 0.008, 0.017, 0.114
----------


In [ ]:
test_labels_femina3_dicece.pth
dice: 0.6978
dices: 0.990, 0.873, 0.635, 0.294 // stdss: 0.012, 0.152, 0.112, 0.125
----------
test_labels_femina3_gdice.pth
dice: 0.6786
dices: 0.989, 0.861, 0.622, 0.243 // stdss: 0.012, 0.150, 0.111, 0.140
----------
test_labels_femina3_gdicefoc.pth
dice: 0.6736
dices: 0.989, 0.856, 0.608, 0.241 // stdss: 0.012, 0.149, 0.108, 0.140
----------
test_labels_femina3_unetr_dicece.pth
dice: 0.7129
dices: 0.990, 0.884, 0.651, 0.327 // stdss: 0.012, 0.154, 0.113, 0.116
----------



test_labels_femina3_dicece.pth
dice: 0.7037
dices: 0.997, 0.892, 0.642, 0.284
stdss: 0.001, 0.156, 0.113, 0.108
----------
test_labels_femina3_gdice.pth
dice: 0.6795
dices: 0.997, 0.887, 0.616, 0.218
stdss: 0.001, 0.155, 0.109, 0.111
----------
test_labels_femina3_gdicefoc.pth
dice: 0.6819
dices: 0.998, 0.893, 0.617, 0.219
stdss: 0.001, 0.156, 0.109, 0.111
----------
test_labels_femina3_unetr_dicece.pth
dice: 0.7293
dices: 0.998, 0.907, 0.668, 0.345
stdss: 0.001, 0.158, 0.116, 0.127

In [ ]:
#monai.config.deviceconfig.print_config()

In [113]:
from glob import glob
import os
import torchmetrics
test_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'LabelsBackFromAtlas', '*_id_fromunetr.nii.gz')))


ants_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'Labels', '*_id.nii.gz')))
opsy_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'LabelsBackFromAtlas', '*_id_fromop_ants.nii.gz')))

#dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)           
dice_metric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=4)

c_1 = []
c_2 = []
c_3 = []
c_4 = []
for i in range(len(test_labels)):
    mouseid = "_".join(test_labels[i].split('/')[-1].split('_')[0:2])
    a = [s for s in ants_labels if mouseid in s][0]
    b = [s for s in opsy_labels if mouseid in s][0]
    ants_lbl = torch.from_numpy(nib.load(a).get_fdata()).ravel().type('torch.LongTensor')
    opsy_lbl = torch.from_numpy(nib.load(b).get_fdata()).ravel().type('torch.LongTensor')
    mask = (opsy_lbl != 0)
    ants_lbl = ants_lbl * mask
    
    dice = dice_metric(opsy_lbl, ants_lbl)
    c_1.append(dice[0])
    c_2.append(dice[1])
    c_3.append(dice[2])
    c_4.append(dice[3])
    
for i in [c_1,c_2,c_3,c_4]:
    print("{:.3f} - {:.3f}".format(np.mean(i),np.std(i)))  

0.994 - 0.010
0.757 - 0.132
0.233 - 0.038
0.010 - 0.004


In [114]:
from glob import glob
import os
import torchmetrics
test_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'LabelsBackFromAtlas', '*_id_fromunetr.nii.gz')))


ants_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'Labels', '*_id.nii.gz')))
opsy_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'LabelsBackFromAtlas', '*_id_fromsyn_ants.nii.gz')))

#dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)           
dice_metric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=4)

c_1 = []
c_2 = []
c_3 = []
c_4 = []
for i in range(len(test_labels)):
    mouseid = "_".join(test_labels[i].split('/')[-1].split('_')[0:2])
    a = [s for s in ants_labels if mouseid in s][0]
    b = [s for s in opsy_labels if mouseid in s][0]
    ants_lbl = torch.from_numpy(nib.load(a).get_fdata()).ravel().type('torch.LongTensor')
    opsy_lbl = torch.from_numpy(nib.load(b).get_fdata()).ravel().type('torch.LongTensor')
    mask = (opsy_lbl != 0)
    ants_lbl = ants_lbl * mask
    
    dice = dice_metric(opsy_lbl, ants_lbl)
    c_1.append(dice[0])
    c_2.append(dice[1])
    c_3.append(dice[2])
    c_4.append(dice[3])
    
for i in [c_1,c_2,c_3,c_4]:
    print("{:.3f} - {:.3f}".format(np.mean(i),np.std(i)))

0.991 - 0.011
0.739 - 0.129
0.220 - 0.035
0.007 - 0.003


In [120]:
from glob import glob
import os
import torchmetrics
test_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'LabelsBackFromAtlas', '*_id_fromunetr.nii.gz')))


ants_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'Labels', '*_id.nii.gz')))
opsy_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'LabelsBackFromAtlas', '*_id_fromsyn_spm_c1c2c3.nii.gz')))

#dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)           
dice_metric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=4)

c_1 = []
c_2 = []
c_3 = []
c_4 = []
for i in range(len(test_labels)):
    mouseid = "_".join(test_labels[i].split('/')[-1].split('_')[0:2])
    a = [s for s in ants_labels if mouseid in s][0]
    b = [s for s in opsy_labels if mouseid in s][0]
    ants_lbl = torch.from_numpy(nib.load(a).get_fdata()).ravel().type('torch.LongTensor')
    opsy_lbl = torch.from_numpy(nib.load(b).get_fdata()).ravel().type('torch.LongTensor')
    mask = (opsy_lbl != 0)
    ants_lbl = ants_lbl * mask
    
    dice = dice_metric(opsy_lbl, ants_lbl)
    c_1.append(dice[0])
    c_2.append(dice[1])
    c_3.append(dice[2])
    c_4.append(dice[3])
    
for i in [c_1,c_2,c_3,c_4]:
    print("{:.3f} - {:.3f}".format(np.mean(i),np.std(i)))

0.986 - 0.011
0.787 - 0.138
0.647 - 0.112
0.060 - 0.023


In [121]:
from glob import glob
import os
import torchmetrics
test_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'LabelsBackFromAtlas', '*_id_fromunetr.nii.gz')))


ants_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'Labels', '*_id.nii.gz')))
opsy_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'LabelsBackFromAtlas', '*_id_fromop_spm_c1c2c3.nii.gz')))

#dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)           
dice_metric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=4)

c_1 = []
c_2 = []
c_3 = []
c_4 = []
for i in range(len(test_labels)):
    mouseid = "_".join(test_labels[i].split('/')[-1].split('_')[0:2])
    a = [s for s in ants_labels if mouseid in s][0]
    b = [s for s in opsy_labels if mouseid in s][0]
    ants_lbl = torch.from_numpy(nib.load(a).get_fdata()).ravel().type('torch.LongTensor')
    opsy_lbl = torch.from_numpy(nib.load(b).get_fdata()).ravel().type('torch.LongTensor')
    mask = (opsy_lbl != 0)
    ants_lbl = ants_lbl * mask
    
    dice = dice_metric(opsy_lbl, ants_lbl)
    c_1.append(dice[0])
    c_2.append(dice[1])
    c_3.append(dice[2])
    c_4.append(dice[3])
    
for i in [c_1,c_2,c_3,c_4]:
    print("{:.3f} - {:.3f}".format(np.mean(i),np.std(i)))

0.992 - 0.011
0.821 - 0.143
0.701 - 0.119
0.138 - 0.039


In [ ]:
t = 0.01 atlas
0.9751 - 0.0060
0.7725 - 0.0682
0.6227 - 0.0551
0.0487 - 0.0239

In [21]:
from glob import glob
import os
import torchmetrics
ants_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'Labels2', '*_id.nii.gz')))
opsy_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'LabelsBackFromAtlas', '*_id_fromsyn_spm.nii.gz')))

print(len(ants_labels))
print(len(opsy_labels))

#dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)           
dice_metric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=4)

c_1 = []
c_2 = []
c_3 = []
c_4 = []
for i in range(len(ants_labels)):
    ants_lbl = torch.from_numpy(nib.load(ants_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    opsy_lbl = torch.from_numpy(nib.load(opsy_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    dice = dice_metric(opsy_lbl, ants_lbl)
    c_1.append(dice[0])
    c_2.append(dice[1])
    c_3.append(dice[2])
    c_4.append(dice[3])
    
for i in [c_1,c_2,c_3,c_4]:
    print("{:.4f} - {:.4f}".format(np.mean(i),np.std(i)))

135
135
0.9839 - 0.0060
0.8026 - 0.0709
0.6391 - 0.0566
0.0260 - 0.0117


In [ ]:
t = 0.10 atlas
0.9880 - 0.0059
0.8353 - 0.0725
0.6888 - 0.0592
0.0824 - 0.0164

t = 0.01 atlas
0.9826 - 0.0059
0.8092 - 0.0703
0.6758 - 0.0576
0.1058 - 0.0307


In [10]:
from glob import glob
import os
import torchmetrics

ants_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'LabelsBackFromAtlas', '*_id.nii.gz')))
opsy_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'LabelsBackFromAtlas', '*_id_fromsyn_spm.nii.gz')))

print(len(ants_labels))
print(len(opsy_labels))

#dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)           
dice_metric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=4)

c_1 = []
c_2 = []
c_3 = []
c_4 = []
for i in range(len(ants_labels)):
    ants_lbl = torch.from_numpy(nib.load(ants_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    opsy_lbl = torch.from_numpy(nib.load(opsy_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    dice = dice_metric(opsy_lbl, ants_lbl)
    c_1.append(dice[0])
    c_2.append(dice[1])
    c_3.append(dice[2])
    c_4.append(dice[3])
    
for i in [c_1,c_2,c_3,c_4]:
    print("{:.4f} - {:.4f}".format(np.mean(i),np.std(i)))

0
135
nan - nan
nan - nan
nan - nan
nan - nan


In [23]:
from glob import glob
import os
opsy_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'LabelsBackFromAtlas', '*_id_fromsyn.nii.gz')))
opsy_labels = sorted(glob(os.path.join('dataset3', 'Femina3', 'LabelsBackFromAtlas', '*_id_fromop.nii.gz')))

print(len(ants_labels))
print(len(opsy_labels))

#dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)           
dice_metric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=4)

c_1 = []
c_2 = []
c_3 = []
c_4 = []
for i in range(len(ants_labels)):
    ants_lbl = torch.from_numpy(nib.load(ants_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    opsy_lbl = torch.from_numpy(nib.load(opsy_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    dice = dice_metric(opsy_lbl, ants_lbl)
    c_1.append(dice[0])
    c_2.append(dice[1])
    c_3.append(dice[2])
    c_4.append(dice[3])
    
for i in [c_1,c_2,c_3,c_4]:
    print("{:.4f} - {:.4f}".format(np.mean(i),np.std(i)))

135
135
0.9882 - 0.0056
0.7606 - 0.0662
0.2251 - 0.0199
0.0054 - 0.0021


In [142]:
from glob import glob
import os
import torchmetrics
test_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id_fromunetr.nii.gz')))


ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id.nii.gz')))
opsy_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id_fromop2.nii.gz')))

#dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)           
dice_metric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=4)

c_1 = []
c_2 = []
c_3 = []
c_4 = []
for i in range(len(test_labels)):
    mouseid = test_labels[i].split('/')[-1].split('_')[0]
    a = [s for s in ants_labels if mouseid in s][0]
    b = [s for s in opsy_labels if mouseid in s][0]
    ants_lbl = torch.from_numpy(nib.load(a).get_fdata()).ravel().type('torch.LongTensor')
    opsy_lbl = torch.from_numpy(nib.load(b).get_fdata()).ravel().type('torch.LongTensor')
    mask = (opsy_lbl != 0)
    ants_lbl = ants_lbl * mask
    
    dice = dice_metric(opsy_lbl, ants_lbl)
    c_1.append(dice[0])
    c_2.append(dice[1])
    c_3.append(dice[2])
    c_4.append(dice[3])
    
for i in [c_1,c_2,c_3,c_4]:
    print("{:.3f} - {:.3f}".format(np.mean(i),np.std(i)))
    

0.999 - 0.000
0.775 - 0.016
0.979 - 0.001
0.646 - 0.015


In [143]:
from glob import glob
import os
import torchmetrics
test_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id_fromunetr.nii.gz')))


ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id.nii.gz')))
opsy_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id_fromdl.nii.gz')))

#dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)           
dice_metric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=4)

c_1 = []
c_2 = []
c_3 = []
c_4 = []
for i in range(len(test_labels)):
    mouseid = test_labels[i].split('/')[-1].split('_')[0]
    a = [s for s in ants_labels if mouseid in s][0]
    b = [s for s in opsy_labels if mouseid in s][0]
    ants_lbl = torch.from_numpy(nib.load(a).get_fdata()).ravel().type('torch.LongTensor')
    opsy_lbl = torch.from_numpy(nib.load(b).get_fdata()).ravel().type('torch.LongTensor')
    mask = (opsy_lbl != 0)
    ants_lbl = ants_lbl * mask
    
    dice = dice_metric(opsy_lbl, ants_lbl)
    c_1.append(dice[0])
    c_2.append(dice[1])
    c_3.append(dice[2])
    c_4.append(dice[3])
    
for i in [c_1,c_2,c_3,c_4]:
    print("{:.3f} - {:.3f}".format(np.mean(i),np.std(i)))
    

0.999 - 0.000
0.758 - 0.015
0.978 - 0.001
0.630 - 0.027


In [145]:
from glob import glob
import os
import torchmetrics
test_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id_fromunetr.nii.gz')))


ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id.nii.gz')))
opsy_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id_frompair2.nii.gz')))

#dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)           
dice_metric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=4)

c_1 = []
c_2 = []
c_3 = []
c_4 = []
for i in range(len(test_labels)):
    mouseid = test_labels[i].split('/')[-1].split('_')[0]
    a = [s for s in ants_labels if mouseid in s][0]
    b = [s for s in opsy_labels if mouseid in s][0]
    ants_lbl = torch.from_numpy(nib.load(a).get_fdata()).ravel().type('torch.LongTensor')
    opsy_lbl = torch.from_numpy(nib.load(b).get_fdata()).ravel().type('torch.LongTensor')
    mask = (opsy_lbl != 0)
    ants_lbl = ants_lbl * mask
    
    dice = dice_metric(opsy_lbl, ants_lbl)
    c_1.append(dice[0])
    c_2.append(dice[1])
    c_3.append(dice[2])
    c_4.append(dice[3])
    
for i in [c_1,c_2,c_3,c_4]:
    print("{:.3f} - {:.3f}".format(np.mean(i),np.std(i)))
    

0.999 - 0.000
0.767 - 0.018
0.979 - 0.001
0.630 - 0.031


In [140]:
from glob import glob
import os
import torchmetrics
import nibabel as nib
ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id.nii.gz')))
opsy_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id_fromop.nii.gz')))

#dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)           
dice_metric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=4)

c_1 = []
c_2 = []
c_3 = []
c_4 = []
for i in range(len(ants_labels)):
    ants_lbl = torch.from_numpy(nib.load(ants_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    opsy_lbl = torch.from_numpy(nib.load(opsy_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    dice = dice_metric(opsy_lbl, ants_lbl)
    c_1.append(dice[0])
    c_2.append(dice[1])
    c_3.append(dice[2])
    c_4.append(dice[3])
    
for i in [c_1,c_2,c_3,c_4]:
    print("{:.4f} - {:.4f}".format(np.mean(i),np.std(i)))

KeyboardInterrupt: 

In [11]:
from glob import glob
import os
import torchmetrics
ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id.nii.gz')))
opsy_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id_fromop.nii.gz')))

print(len(ants_labels))
print(len(opsy_labels))


#dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)           
dice_metric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=4)

c_1 = []
c_2 = []
c_3 = []
c_4 = []
for i in range(len(ants_labels)):
    ants_lbl = torch.from_numpy(nib.load(ants_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    opsy_lbl = torch.from_numpy(nib.load(opsy_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    dice = dice_metric(opsy_lbl, ants_lbl)
    c_1.append(dice[0])
    c_2.append(dice[1])
    c_3.append(dice[2])
    c_4.append(dice[3])
    
for i in [c_1,c_2,c_3,c_4]:
    print("{:.4f} - {:.4f}".format(np.mean(i),np.std(i)))

27
27
0.9897 - 0.0014
0.5825 - 0.0788
0.9222 - 0.0123
0.4605 - 0.1378


In [8]:
from glob import glob
import os
import torchmetrics
ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id.nii.gz')))
opsy_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id_fromdl.nii.gz')))

#dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)           
dice_metric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=4)

c_1 = []
c_2 = []
c_3 = []
c_4 = []
for i in range(len(ants_labels)):
    ants_lbl = torch.from_numpy(nib.load(ants_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    opsy_lbl = torch.from_numpy(nib.load(opsy_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    dice = dice_metric(opsy_lbl, ants_lbl)
    c_1.append(dice[0])
    c_2.append(dice[1])
    c_3.append(dice[2])
    c_4.append(dice[3])
    
for i in [c_1,c_2,c_3,c_4]:
    print("{:.4f} - {:.4f}".format(np.mean(i),np.std(i)))

0.9897 - 0.0012
0.5926 - 0.0636
0.9210 - 0.0106
0.4789 - 0.1224


In [7]:
from glob import glob
import os
import torchmetrics
ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id.nii.gz')))
opsy_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id_frompair2.nii.gz')))

print(len(ants_labels))
print(len(opsy_labels))


#dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)           
dice_metric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=4)

c_1 = []
c_2 = []
c_3 = []
c_4 = []
for i in range(len(ants_labels)):
    ants_lbl = torch.from_numpy(nib.load(ants_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    opsy_lbl = torch.from_numpy(nib.load(opsy_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    dice = dice_metric(opsy_lbl, ants_lbl)
    c_1.append(dice[0])
    c_2.append(dice[1])
    c_3.append(dice[2])
    c_4.append(dice[3])
    
for i in [c_1,c_2,c_3,c_4]:
    print("{:.4f} - {:.4f}".format(np.mean(i),np.std(i)))

27
27
0.9895 - 0.0015
0.5854 - 0.0756
0.9199 - 0.0126
0.4605 - 0.1335


In [6]:
from glob import glob
import os
import torchmetrics
ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id.nii.gz')))
opsy_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id_fromop2.nii.gz')))

print(len(ants_labels))
print(len(opsy_labels))


#dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)           
dice_metric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=4)

c_1 = []
c_2 = []
c_3 = []
c_4 = []
for i in range(len(ants_labels)):
    ants_lbl = torch.from_numpy(nib.load(ants_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    opsy_lbl = torch.from_numpy(nib.load(opsy_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    dice = dice_metric(opsy_lbl, ants_lbl)
    c_1.append(dice[0])
    c_2.append(dice[1])
    c_3.append(dice[2])
    c_4.append(dice[3])
    
for i in [c_1,c_2,c_3,c_4]:
    print("{:.4f} - {:.4f}".format(np.mean(i),np.std(i)))

27
27
0.9893 - 0.0016
0.5792 - 0.0734
0.9185 - 0.0146
0.4772 - 0.1428


In [12]:
from glob import glob
import os
import torchmetrics
ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id_fromop.nii.gz')))
opsy_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'LabelsBackFromAtlas', '*_id_frompair.nii.gz')))

#dice_metric = DiceMetric(include_background=True, reduction="mean_channel", get_not_nans=False)           
dice_metric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=4)

c_1 = []
c_2 = []
c_3 = []
c_4 = []
for i in range(len(ants_labels)):
    ants_lbl = torch.from_numpy(nib.load(ants_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    opsy_lbl = torch.from_numpy(nib.load(opsy_labels[i]).get_fdata()).ravel().type('torch.LongTensor')
    dice = dice_metric(opsy_lbl, ants_lbl)
    c_1.append(dice[0])
    c_2.append(dice[1])
    c_3.append(dice[2])
    c_4.append(dice[3])
    
for i in [c_1,c_2,c_3,c_4]:
    print("{:.4f} - {:.4f}".format(np.mean(i),np.std(i)))

27
27
0.9971 - 0.0005
0.8540 - 0.0397
0.9741 - 0.0049
0.7553 - 0.0311
